In [1]:
class Option:
    def __init__(self,**kargs):
        for k in kargs:
            if (k == 'S'):
                self.S = kargs[k]
            elif(k == 'K'):
                self.K = kargs[k]
            elif(k == 'r'):
                self.r = kargs[k]
            elif(k == 'sigma'):
                self.sigma = kargs[k]
            elif(k == 't'):
                self.t = kargs[k]
            elif(k == 'd'):
                self.d = kargs[k]
            elif(k == 'price'):
                self.price = kargs[k]
            elif(k == 'typ'):
                self.typ = kargs[k]
            else:
                print('Please input key work arguments as S,K,r,sigma,t,d,price')
                raise 
    
    def calculateIV(self,s0):
        import sympy as sy
        sigma = sy.Symbol('sigma')
        d1 = (sy.log(S/K)+(r-d+(sigma*sigma)/2)*t)/(sigma*sy.sqrt(t))
        d2 = d1-sigma*sy.sqrt(t)
        
        if (self.typ =='c'):
            bsp  = S*sy.exp(-d*t)*(0.5*(1+sy.erf(d1/sy.sqrt(2)))) - K*sy.exp(-r*t)*(0.5*(1+sy.erf(d2/sy.sqrt(2))))
        elif (self.typ == 'p'):
            bsp = K*sy.exp(-r*t)*(0.5*(1+sy.erf(-d2/sy.sqrt(2)))) - S*sy.exp(-d*t)*(0.5*(1+sy.erf(-d1/sy.sqrt(2))))
            
        res = sy.nsolve(bsp-self.price,s0)
        return float(res)


if __name__ == '__main__':
    
    import csv
    
    feature = {}
    cbid= []
    cask = []
    K_list = []
    pbid = []
    pask = []
    read_in =[cbid,cask,K_list,pbid,pask]
    with open('AMZN_mar20_opt_quotes_20200220.csv','r') as f:
        raw = list(csv.reader(f))
    
    for c in range(0,len(raw)):
        row = raw[c]
        if (3<= c <=6):
            feature[row[0]]=float(row[1])
        elif (c>7 and row[0] !=''):
            for i in range(0,5):
                read_in[i].append(float(row[i]))
        
    
    t = feature['T']
    S = feature['StockPrice']
    d = feature['Dividend']
    r = feature['RiskFree']
    
    call_vol = []
    put_vol = []

        
    for i in range(0,len(K_list)):
        K = K_list[i]
        cp = (cask[i]+cbid[i])/2
        pp = (pask[i]+pbid[i])/2
        tmpc = Option(t=t,S =S,d=d,r=r,K=K,price =cp,typ = 'c')
        tmpp = Option(t=t,S =S,d=d,r=r,K=K,price =pp,typ = 'p')
        call_vol.append(tmpc.calculateIV(0.1))
        put_vol.append(tmpp.calculateIV(0.1))
    
    res = read_in +[call_vol]+[put_vol]
    res_o =[[res[i][j] for i in range(0,7)] for j in range(0,51)]
    with open('output.csv','w') as f1:
        writer = csv.writer(f1)
        writer.writerow(['Bid','Ask','StrikePrice','Bid','Ask','CallVol','PutVol'])
        for i in range(0,len(res_o)):
            writer.writerow(res_o[i])